In [18]:
import torch
if torch.cuda.is_available():
    torch.cuda.init()
    assert torch.cuda.is_initialized()
    print('Cuda is ready')    
    
import os
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from SAITS_functions import *

Cuda is ready


In [19]:
dataset_path =  f"{os.getcwd()}/data"

file_path = os.path.join(dataset_path, 'datasets.h5')
with h5py.File(file_path, 'r') as hf:  # read data from h5 file
    #X = hf['train']['X'][:]
    X0 = hf['train']['X'][:]
print(
X0.ndim,
X0.shape,
X0.size, dataset_path)

3 (15, 9, 51) 6885 /DeepenData/Repos/Tyrosinemia/data


In [20]:

from functions_aa import *
args, model_args       = get_args(seq_len = 9, feature_num  = 51, batch_size   = 3)
assert args.dataset_path == dataset_path
masked_imputation_task = True
model_type             = 'SAITS'
args.batch_size        = 3
args.device            = 'cuda'
unified_dataloader     = UnifiedDataLoader(args.dataset_path, args.seq_len, args.feature_num, model_type, args.batch_size, args.num_workers, masked_imputation_task)
train_data, val_data, test_data = unified_dataloader.prepare_all_data_for_imputation()

In [21]:
import pickle

with open('sets_ids_splitted.pickle', 'rb') as handle:
    sets_ids_splitted = pickle.load(handle)    

sets_ids_splitted['train_set_ids']

idx_to_code_dict = dict(zip(range(15), sets_ids_splitted['train_set_ids']))
idx_to_code_dict

{0: 'T07',
 1: 'T08',
 2: 'T19',
 3: 'T13',
 4: 'T15',
 5: 'T11',
 6: 'T10',
 7: 'T17',
 8: 'T04',
 9: 'T16',
 10: 'T01',
 11: 'T02',
 12: 'T12',
 13: 'T09',
 14: 'T14'}

In [22]:
list(train_data).__len__(), list(train_data)[0].__len__(), list(train_data)[0].__len__(), list(train_data)[0][0].shape, list(train_data)[1][1].shape

idxs = []
for i, t, x in train_data:
    idxs.append(i)
    
a_over_sample = torch.cat(idxs).numpy().tolist()

import pickle

def idxs_to_codes(a_over_sample):

    with open('sets_ids_splitted.pickle', 'rb') as handle:
        sets_ids_splitted = pickle.load(handle)    


    idx_to_code_dict = dict(zip(range(15), sets_ids_splitted['train_set_ids']))


    return [idx_to_code_dict[s] for s in a_over_sample]

np.array(idxs_to_codes(a_over_sample))

array(['T04', 'T17', 'T07', 'T09', 'T16', 'T08', 'T10', 'T12', 'T10',
       'T15', 'T09', 'T04', 'T08', 'T16', 'T04', 'T13', 'T04', 'T14',
       'T17', 'T14', 'T04', 'T04', 'T10', 'T04', 'T14', 'T01', 'T08',
       'T02', 'T01', 'T15', 'T11', 'T16', 'T16'], dtype='<U3')

In [23]:
saved_parameters_path = f"{os.getcwd()}/state_dict_best_trained_model.pth"
model = SAITS(**model_args)
model.load_state_dict(torch.load(saved_parameters_path));
model.to('cuda');

model_args

{'device': 'cuda',
 'MIT': True,
 'n_groups': 5,
 'n_group_inner_layers': 1,
 'd_time': 9,
 'd_feature': 51,
 'dropout': 0.0,
 'd_model': 256,
 'd_inner': 512,
 'n_head': 8,
 'd_k': 32,
 'd_v': 32,
 'input_with_mask': True,
 'diagonal_attention_mask': True,
 'param_sharing_strategy': 'inner_group'}

In [24]:
model.to('cuda')
idxs = []
imputations = []


model.eval()
with torch.no_grad():
    for _, data  in enumerate(train_data):
        
        indices, X, missing_mask = map(lambda x: x.to(args.device), data)
        inputs = {'indices': indices, 'X': X, 'missing_mask': missing_mask}
        imputed_data, _ = model.impute(inputs)
        idxs.append(indices)
        imputations.append(imputed_data)
        
import pandas as pd
df_concat      = None
lista_tensores = []
codes          = []

df_concat = None
df        = None
for iteration, d in enumerate(zip(torch.cat(imputations), torch.cat(idxs))):
   
    t, i =  d    
    i    = i.cpu().detach().numpy().tolist()
    df   = pd.DataFrame(t.cpu().detach().numpy())
    df['codigo'] = i
    
    if iteration == 1 :       
        #print(9)
        df_concat = pd.concat([df_concat, df], axis=0)
        codes.append(i)
        
    else:
        if i not in codes:
            df_concat = pd.concat([df_concat, df], axis=0)
            codes.append(i)
            
    assert (df_concat.to_numpy() == df_concat.drop_duplicates().to_numpy()).all()      
df_concat

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,codigo
0,3.223391,23.882654,-3.848682,31.236229,260.562134,22.372665,22.017416,1.420000,18.100000,1.320000,...,-4.170576,1.685041,3.203002,-2.277948,-0.904545,-1.234829,2.037983,4.911880,6.717999,10
1,1.940000,35.349998,0.000000,31.299999,249.699997,24.000000,10.400000,1.220000,15.700000,0.900000,...,-168.822968,-40.799358,91.784561,11.853781,73.558624,7.526054,-41.499767,-22.180084,-88.848068,10
2,1.190000,48.799999,-1.749172,23.990891,233.688293,21.336906,17.270130,3.944505,19.918402,0.490000,...,-3.559803,0.155126,2.166530,0.439797,-3.217508,-1.256569,-1.549799,3.774678,1.258294,10
3,1.120000,38.400002,-3.351104,20.180000,202.210007,8.620000,23.458237,1.615403,10.244694,1.654548,...,-2.435750,-0.203902,-0.485598,0.113348,-1.230272,0.864760,-2.219999,2.085447,-4.163616,10
4,1.465417,12.580000,-2.778750,17.090000,280.750000,19.900000,14.108040,2.593686,30.000000,0.590000,...,-2.365386,0.423376,0.885073,-0.258852,-2.085128,-0.636269,-2.133710,4.897211,0.369368,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0.652000,34.400002,0.000000,36.299999,252.800003,29.600000,8.540000,1.000000,13.300000,0.660000,...,0.240000,0.030000,0.020000,0.090000,0.040000,0.040000,0.590000,0.380000,0.010000,1
5,0.620000,21.670000,0.250000,18.320000,428.149994,37.779999,11.332716,1.020000,13.600000,0.740000,...,0.200000,0.030000,0.020000,0.010000,0.030000,0.040000,0.580000,0.340000,0.040000,1
6,0.610000,25.430000,0.070000,18.740000,621.539978,32.619999,19.053955,1.060000,14.000000,0.510000,...,0.240000,0.040000,0.010000,0.030000,0.030000,0.050000,0.670000,0.400000,0.030000,1
7,0.630000,24.410000,0.080000,26.812080,372.523010,34.569828,17.910152,1.080000,14.500000,0.720000,...,0.120000,0.010000,0.010000,0.020000,0.010000,0.040000,0.380000,0.280000,0.010000,1


In [25]:
imputed_train_df = df_concat.copy()  

imputed_train_df['codigo'] = idxs_to_codes(df_concat['codigo'])
imputed_train_df

imputed_train_df.to_csv('imputed_train_df.csv')

In [26]:
model.to('cuda')
idxs = []
imputations = []


model.eval()
with torch.no_grad():
    for _, data  in enumerate(val_data):
        
        indices, X, missing_mask = map(lambda x: x.to(args.device), data)
        inputs = {'indices': indices, 'X': X, 'missing_mask': missing_mask}
        imputed_data, _ = model.impute(inputs)
        idxs.append(indices)
        imputations.append(imputed_data)
        
import pandas as pd
df_concat      = None
lista_tensores = []
codes          = []

df_concat = None
df        = None
for iteration, d in enumerate(zip(torch.cat(imputations), torch.cat(idxs))):
   
    t, i =  d    
    i    = i.cpu().detach().numpy().tolist()
    df   = pd.DataFrame(t.cpu().detach().numpy())
    df['codigo'] = i
    
    if iteration == 1 :       
        #print(9)
        df_concat = pd.concat([df_concat, df], axis=0)
        codes.append(i)
        
    else:
        if i not in codes:
            df_concat = pd.concat([df_concat, df], axis=0)
            codes.append(i)
            
    assert (df_concat.to_numpy() == df_concat.drop_duplicates().to_numpy()).all()      
df_concat

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,codigo
0,0.697000,14.320000,0.120000,21.500000,667.700012,45.200001,14.770000,1.040000,13.700000,0.370000,...,0.110000,0.070000,0.030000,0.070000,0.030000,0.080000,0.350000,0.180000,0.020000,1
1,0.688000,21.570000,0.104600,17.600000,666.299988,39.099998,17.040001,1.150000,14.900000,0.430000,...,0.120000,0.110000,0.050000,0.090000,0.030000,0.230000,0.360000,0.210000,0.090000,1
2,0.690000,15.810000,0.204200,22.900000,910.599976,48.500000,18.780001,1.110000,14.400000,0.270000,...,0.070000,0.050000,0.030000,0.070000,0.020000,0.080000,0.310000,0.230000,0.010000,1
3,0.660000,23.209999,0.108700,21.000000,605.900024,49.400002,12.270000,1.110000,14.400000,0.510000,...,-2.445126,0.746149,-0.348036,-0.529500,-1.521926,-1.227969,-2.712385,4.124435,-2.473349,1
4,0.735000,36.099998,-2.731231,27.600000,505.000000,51.799999,9.750000,1.300000,13.700000,0.390000,...,0.050000,0.010000,0.010000,0.070000,0.010000,0.050000,0.270000,0.150000,0.010000,1
5,0.830000,18.790001,0.040000,13.980000,287.290009,26.629999,10.788209,31.120001,13.900000,0.430000,...,0.060000,0.010000,0.010000,0.080000,0.020000,0.040000,0.360000,0.260000,0.010000,1
6,0.000000,9.050000,0.090000,13.040000,254.220001,22.020000,11.544959,1.100000,14.400000,0.430000,...,0.040000,0.030000,0.020000,0.070000,0.030000,0.050000,0.330000,0.210000,0.010000,1
7,0.800000,15.810000,-3.336619,11.250000,366.709991,26.250000,13.969905,1.070000,14.400000,0.430000,...,0.050000,0.010000,0.020000,0.080000,0.030000,0.050000,0.320000,0.200000,0.010000,1
8,2.019675,25.899963,-3.015741,26.097944,347.494781,35.380581,20.885962,1.160692,14.677876,3.935689,...,0.070000,0.010000,0.020000,0.050000,0.030000,0.050000,0.270000,0.180000,0.020000,1
0,1.020000,0.560000,2.970000,27.200001,721.200012,56.299999,12.810000,2.431387,16.785576,1.780000,...,0.180000,0.090000,0.030000,0.080000,0.060000,0.140000,0.730000,0.450000,0.040000,2


In [27]:
idx_to_code_dict_val_set_ids = dict(zip(range(15), sets_ids_splitted['val_set_ids']))
idx_to_code_dict_val_set_ids

{0: 'T03', 1: 'T05', 2: 'T06', 3: 'T18'}

In [28]:
def idxs_to_codes_val_set_ids(a_over_sample):

    with open('sets_ids_splitted.pickle', 'rb') as handle:
        sets_ids_splitted = pickle.load(handle)    


    idx_to_code_dict = dict(zip(range(15), sets_ids_splitted['val_set_ids']))


    return [idx_to_code_dict[s] for s in a_over_sample]


imputed_val_df = df_concat.copy()  

imputed_val_df['codigo'] = idxs_to_codes_val_set_ids(df_concat['codigo'])
imputed_val_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,codigo
0,0.697000,14.320000,0.120000,21.500000,667.700012,45.200001,14.770000,1.040000,13.700000,0.370000,...,0.110000,0.070000,0.030000,0.070000,0.030000,0.080000,0.350000,0.180000,0.020000,T05
1,0.688000,21.570000,0.104600,17.600000,666.299988,39.099998,17.040001,1.150000,14.900000,0.430000,...,0.120000,0.110000,0.050000,0.090000,0.030000,0.230000,0.360000,0.210000,0.090000,T05
2,0.690000,15.810000,0.204200,22.900000,910.599976,48.500000,18.780001,1.110000,14.400000,0.270000,...,0.070000,0.050000,0.030000,0.070000,0.020000,0.080000,0.310000,0.230000,0.010000,T05
3,0.660000,23.209999,0.108700,21.000000,605.900024,49.400002,12.270000,1.110000,14.400000,0.510000,...,-2.445126,0.746149,-0.348036,-0.529500,-1.521926,-1.227969,-2.712385,4.124435,-2.473349,T05
4,0.735000,36.099998,-2.731231,27.600000,505.000000,51.799999,9.750000,1.300000,13.700000,0.390000,...,0.050000,0.010000,0.010000,0.070000,0.010000,0.050000,0.270000,0.150000,0.010000,T05
5,0.830000,18.790001,0.040000,13.980000,287.290009,26.629999,10.788209,31.120001,13.900000,0.430000,...,0.060000,0.010000,0.010000,0.080000,0.020000,0.040000,0.360000,0.260000,0.010000,T05
6,0.000000,9.050000,0.090000,13.040000,254.220001,22.020000,11.544959,1.100000,14.400000,0.430000,...,0.040000,0.030000,0.020000,0.070000,0.030000,0.050000,0.330000,0.210000,0.010000,T05
7,0.800000,15.810000,-3.336619,11.250000,366.709991,26.250000,13.969905,1.070000,14.400000,0.430000,...,0.050000,0.010000,0.020000,0.080000,0.030000,0.050000,0.320000,0.200000,0.010000,T05
8,2.019675,25.899963,-3.015741,26.097944,347.494781,35.380581,20.885962,1.160692,14.677876,3.935689,...,0.070000,0.010000,0.020000,0.050000,0.030000,0.050000,0.270000,0.180000,0.020000,T05
0,1.020000,0.560000,2.970000,27.200001,721.200012,56.299999,12.810000,2.431387,16.785576,1.780000,...,0.180000,0.090000,0.030000,0.080000,0.060000,0.140000,0.730000,0.450000,0.040000,T06


In [29]:
imputed_val_df.to_csv('imputed_val_df.csv')